#### use the pre-trained biomedgpt from huggingface -- PanaceaAI/BiomedGPT-Base-Pretrained

In [1]:
# # download the same-size pretrained HF variant of the finetuned variant that you want to convert from.
model_name = "BiomedGPT-Base-Pretrained"
# model_name = "instruct-biomedgpt-base" # also tried this but didn't work as well
# !git clone https://huggingface.co/PanaceaAI/{model_name}

In [2]:
# !git clone --single-branch --branch feature/add_transformers https://github.com/OFA-Sys/OFA.git

# !pip install OFA/transformers

Cloning into 'OFA'...
remote: Enumerating objects: 5745, done.
remote: Counting objects: 100% (916/916), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 5745 (delta 695), reused 662 (delta 662), pack-reused 4829 (from 1)
Receiving objects: 100% (5745/5745), 97.78 MiB | 34.77 MiB/s, done.
Resolving deltas: 100% (2243/2243), done.
Processing ./OFA/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 6.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3916872 sha256=c4cfb37907bb6a6c159863aac4aafd4d526ed50140c5d8add15b969b9dad20e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-x0wpj5sr/wheels/76/d5/dd/78c4549cb6e4918e4e5790e271235873c6a3d18920a8ce4d3f
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: tra

In [4]:
model_name

'BiomedGPT-Base-Pretrained'

In [9]:
from transformers import OFATokenizer, OFAModel
tokenizer = OFATokenizer.from_pretrained('BiomedGPT-Base-Pretrained')
model = OFAModel.from_pretrained('BiomedGPT-Base-Pretrained')


BiomedGPT-Base-Pretrained
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


In [10]:
import re

import torch
from PIL import Image
from torchvision import transforms


mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 480

patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


In [7]:
import glob
from pathlib import Path

root_dir = Path('/well/papiez/users/hri611/python/foundation-models-radiology')
jpg_paths = glob.glob(str(root_dir / 'PTX Head to Head Study Data' / '**/*.jpg'), recursive=True)

In [ ]:
# !gdown --id 1kojeqtJSx3MzdY3jd--XzeJrdyiFuzC8 -O example.jpg
import tqdm
results_list = []
for path in tqdm.tqdm(jpg_paths):
    img = Image.open(path)
    # img = Image.open('1200px-Normal_lateral_chest_radiograph_(X-ray).jpg')

    txt = "Does this chest X-ray contain a pneumothorax?"
    inputs = tokenizer([txt], return_tensors="pt").input_ids
    patch_img = patch_resize_transform(img).unsqueeze(0)

    gen = model.generate(inputs, patch_images=patch_img, num_beams=5, no_repeat_ngram_size=3, max_length=16)
    results = tokenizer.batch_decode(gen, skip_special_tokens=True)

    result = results[0]
    result = re.sub(r'[^\w\s]', '', result).strip()

    print(result)

    results_list.append(result)

  0%|▏                                                                                               | 1/413 [00:10<1:13:42, 10.74s/it]

figure reprinted from siah medical school of medicine san diego


  0%|▍                                                                                                 | 2/413 [00:17<56:26,  8.24s/it]

code_1126  a month prior to death


  1%|▋                                                                                                 | 3/413 [00:26<58:34,  8.57s/it]

...

Lung


100%|███████████████████████████████████████████████████████████████████████████████████████████████▊| 412/413 [59:54<00:08,  8.03s/it]

easily seen involving the shoulders and chest


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 413/413 [1:00:00<00:00,  8.72s/it]

Lung


In [ ]:
import pandas as pd
biomedgpt_df = pd.DataFrame({'image_path': jpg_paths, 'response': results_list})
biomedgpt_df['binary_score'] = biomedgpt_df['response'].apply(lambda x: 1 if 'yes' in x.lower() else 0 if 'no' in x.lower() else None)
biomedgpt_df.to_csv('ptx_biomedgpt_scores.csv', index=False)

In [ ]:
biomedgpt_df['binary_score'].value_counts()

binary_score
0.0    139
1.0      7
Name: count, dtype: int64